# Tool Calling

![](2026-01-21-14-17-46.png)

In [10]:
# Setup
import os
import json
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [11]:
user_input = "What's the weather in San Francisco?"
# this is a from scratch example showing how tool calling began in the early days!
response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=256,
    messages=[
        {"role": "user", "content": f"""
         Imagine you are a weather specialist with access to a 'get_weather' function that works like this:
         
         get_weather(location: str) -> str. 
         
         When you the user asks for weather info, you're output shouldl be just the tool call with the right arguments.
         make sure the output tool call is revolved by square brackets like: [get_weather(location="San Francisco")]
         
         Here is the input from the user:
        {user_input}
         """}
    ]
)

response.content

[TextBlock(citations=None, text='[get_weather(location="San Francisco")]', type='text')]

In [12]:
response.content[0].text

'[get_weather(location="San Francisco")]'

In [18]:
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # This is a mock function - in reality, you'd call a weather API
    weather_data = {
        "san francisco": "sunny, 72°F",
        "new york": "cloudy, 65°F",
        "london": "rainy, 58°F",
        "tokyo": "clear, 68°F"
    }
    location_lower = location.lower()
    print('Executing tool call: ', location_lower)
    return weather_data.get(location_lower, f"Weather data not available for {location}")


def execute_tool_call(tool_call: str) -> str:
    """
    Execute a tool call.
    
    Args:
        tool_call: The tool call to execute
        
    Returns:
        The result of the tool call
    """
    # Parse the tool call
    
    
    # Call the function
    # strip the square brackets
    tool_call = tool_call.strip("[]")
    if "get_weather" in tool_call:
        print('Executing tool call: ', tool_call)
        exec(tool_call)
    else:
        raise ValueError(f"Unknown tool: {tool_call}")
    
    return "Tool call executed successfully"

def llm_call(user_input: str) -> str:
    """
    Call the LLM with the user input.
    
    Args:
        user_input: The user input
        
    Returns:
        The response from the LLM
    """
    client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=256,
    messages=[
        {"role": "user", "content": f"""
         Imagine you are a weather specialist with access to a 'get_weather' function that works like this:
         
         get_weather(location: str) -> str. 
         
         When you the user asks for weather info, you're output shouldl be just the tool call with the right arguments.
         make sure the output tool call is revolved by square brackets like: [get_weather(location="San Francisco")]
         
         Here is the input from the user:
        {user_input}
         """}
    ]
)
    
    return response.content[0].text

# Test the tool calling
user_input = "What's the weather in san francisco?"
tool_call = llm_call(user_input)
result = execute_tool_call(tool_call)
print(result)

Executing tool call:  get_weather(location="San Francisco")
Executing tool call:  san francisco
Tool call executed successfully


# .. What could be a use case when we use multiple different LLMs from different vendors in a single Agent? why not using the same LLMs within a single Agent?


MCP!!!!!

# How much more training can you do to a model like claude when it has already been pre trained

YOu can't unless there is an API for fine tunning

# AT: is claude-sonnet-4-5 model that you use is something available in public like ollama that we can run locally? or this kind of nice interaction only available in hosted model? I'm still hazy about how this LLM manages is knowledge base

its a closed source model! You use it under an API!

# so in general it looks like the messages parameter is used to help the LLM know which tool to use. It's more on defining in the messages when it should be used? 

Messages parameter is to keep track of conversation history (user inputs, tool calls, tool outputs. etc....)

We can customize the system message or the prompt to teach and tell the model about the tools, but usually that's already done behind the HOOD!